In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import mysql.connector
import datetime
from openpyxl import Workbook
from openpyxl.styles import PatternFill

#### MYSQL ACCIONES

In [12]:
#Funcion que obtiene los numeros de telefono con menos de 3 incidencias
def obtenerNumeros():
    conexion = mysql.connector.connect(
    host = '127.0.0.1',
    user = 'prueba',
    password = '1234',
    database = 'number_spam'
)
    cursor = conexion.cursor()

    consulta = 'SELECT * FROM NUMERO AS NM WHERE NM.INCIDENCIAS < 3'
    cursor.execute(consulta)
    resultado = cursor.fetchall()
    conexion.close()
    
    return resultado


def ActualizarNumero(numero:str, incidencias:str):
    conexion = mysql.connector.connect(
        host = '127.0.0.1',
        user = 'prueba',
        password = '1234',
        database = 'number_spam'
    )
    cursor = conexion.cursor()
    consulta = 'UPDATE NUMERO SET INCIDENCIAS = %s, ULTIMA_ACTUALIZACION = %s WHERE NUMERO = %s ' 
    
    #Fecha actual
    fecha_actual = datetime.datetime.now()
    timestamp = fecha_actual.strftime('%Y-%m-%d %H:%M:%S')

    cursor.execute(consulta, (int(incidencias), timestamp,numero))
    conexion.commit()
    conexion.close() 


def ObtenerTodo():
    conexion = mysql.connector.connect(
        host = '127.0.0.1',
        user = 'prueba',
        password = '1234',
        database = 'number_spam'
    )
    cursor = conexion.cursor()

    consulta = "SELECT * FROM NUMERO"
    cursor.execute(consulta)
    resultado = cursor.fetchall()

    conexion.commit()
    conexion.close()
    return resultado

#### EXCEL ACCIONES

In [22]:
def GuardarNumerosExcel(lista, ruta):
    wb = Workbook()

    ws = wb.active
    ws.title = "NUMEROS SPAM"

    ws['A1'] = "NUMEROS"
    ws['B1'] = "INCIDENCIAS"

    for i, info in enumerate(lista):

        ws[f'A{i+2}'] = info[0]
        ws[f'B{i+2}'] = info[1]
        
        if int(info[1]) >=3:
            ws[f'A{i+2}'].fill =  PatternFill(start_color='EB455F', end_color='EB455F', fill_type='solid')
            ws[f'B{i+2}'].fill =  PatternFill(start_color='EB455F', end_color='EB455F', fill_type='solid')
        else:
            ws[f'A{i+2}'].fill =  PatternFill(start_color='DDF7E3', end_color='EB455F', fill_type='solid')
            ws[f'B{i+2}'].fill =  PatternFill(start_color='DDF7E3', end_color='EB455F', fill_type='solid')

    wb.save(ruta)
    

Clase que buscara el numero de telefono que le pasemos por parametro en la pagina web lista spam

In [14]:
class BuscarTelefono:

    def __init__(self, numero_telefono:str, driver):
        self.driver = driver
        self.numero_telefono = numero_telefono
        self.wait1 = WebDriverWait(self.driver, 20)



    def BuscarTelefono(self):
        self.driver.get('https://www.listaspam.com/')

        btn_cookie = self.ObtenerClass('css-vm4tgo')
        btn_cookie.click()


        input_search = self.ObtenerXPATH('//*[@id="Telefono"]')
        input_search.click()
        input_search.send_keys(self.numero_telefono)

        btn_search = self.ObtenerXPATH('//*[@id="search_submit"]')
        btn_search.click()

        numero_incidencias = self.ObtenerXPATH('//*[@id="content"]/div/main/section[1]/div[2]/div[3]/span')
        return (self.numero_telefono, numero_incidencias.text)

    def ObtenerXPATH(self, xPath:str):
        return self.wait1.until(EC.presence_of_element_located((
                By.XPATH, xPath)))
    
    def ObtenerClass(self, ClassName:str):
        return self.wait1.until(EC.presence_of_element_located((
                By.CLASS_NAME, ClassName)))

##### Ejecucion del programa

In [15]:
lista_numeros = obtenerNumeros()

driver = webdriver.Chrome()
driver.maximize_window()

#De esta forma no nos apareceria un captcha pero no es una forma optima dado
#que estamos abriendo constantemente un navegador nuevo

lista = []

for numero in lista_numeros:
    driver = webdriver.Chrome()
    driver.maximize_window()
    buscar = BuscarTelefono(numero[0], driver)
    info_numero = buscar.BuscarTelefono()
    ActualizarNumero(info_numero[0], info_numero[1])
    lista.append(info_numero)

#Busqueda de uno en uno
"""
buscar = BuscarTelefono(lista_numeros[1][0], driver)
resultado = buscar.BuscarTelefono()
ActualizarNumero(resultado[0], resultado[1])
"""

driver.close()


Obtiene todos los numeros guardados de la base de datos y los inserta en un archivo excel

In [23]:
lista_numeros = ObtenerTodo()

GuardarNumerosExcel(lista_numeros, 'salidas/numero.xlsx')